In [1]:

import pandas as pd
import copy
import os
import math

把XILINX的pkg.txt文件转换为适合测试管脚输入文件。 

使用前需要把pkg.txt的最后一行去掉，以及把列名字的行中的空格去掉





In [10]:
CLK_PIN="AJ11"
CLK_PIN_P=""
CLK_PIN_N=""
CLK_IO_STANDARD="lvcmos18"
CLK_DIFF_IO_STANDARD="DIFF_HSTL_II_18"
DATA_IO_STANDARD="lvcmos18"
CLK_FREQ=50e6

In [3]:
FILE="xc7vx415tffg1157pkg.txt"
PART_NAME=os.path.splitext(FILE)[0][0:-3]

In [4]:
df=pd.read_csv(FILE,delim_whitespace=True,header=1,dtype=str)
df.sort_values("Bank",inplace=True)
df.fillna("",inplace=True)

In [5]:
pinname2order=list()

In [6]:
fxdc=open("pin.xdc","w")
n=0
for i in df.index:
    sname=df.iloc[i]['PinName']
    pin = df.iloc[i]['Pin']
    if pin==CLK_PIN:
            print(f"set_property PACKAGE_PIN  {pin} [get_ports clk]",file=fxdc)
            print(f"set_property IOSTANDARD  {CLK_IO_STANDARD} [get_ports clk]",file=fxdc)
    elif pin==CLK_PIN_P:
            print(f"set_property PACKAGE_PIN  {pin} [get_ports clk_p]",file=fxdc)
            print(f"set_property IOSTANDARD  {CLK_DIFF_IO_STANDARD} [get_ports clk_p]",file=fxdc)
    elif pin==CLK_PIN_N:
            print(f"set_property PACKAGE_PIN  {pin} [get_ports clk_n]",file=fxdc)
            print(f"set_property IOSTANDARD  {CLK_DIFF_IO_STANDARD} [get_ports clk_n]",file=fxdc)
    elif(len(sname.strip())):
        if sname.startswith("IO_"):
            
            pinname2order.append(pin)
            print(f"set_property PACKAGE_PIN  {pin} [get_ports data[{n}]]",file=fxdc)
            print(f"set_property IOSTANDARD   {DATA_IO_STANDARD} [get_ports data[{n}]]",file=fxdc)
            n=n+1
fxdc.close()    

In [7]:
f=open("mem_init.vh","w")
for index in range(len(pinname2order)):
    sname=f"data[{index}]"
    pin = pinname2order[index]
    #print(ls)
    s=pin
    if(len(s)<4):
        s=s+' '*(4-len(s))
    b="2'b11};"
    
    for i in range(4):
        b="1'b0,"+b
        b="\""+s[i]+"\","+b
        
        
        b="2'b11,"+b
    b="{"+b
    
    print("mem[{0}]={1}".format(index,b),file=f)
f.close()

In [11]:
with open("define.vh","w") as f:
    print(f"`define BAUD_DIV {int(CLK_FREQ/115200-1)}",file=f)
    print(f"`define N_PINS {len(pinname2order)}",file=f)
    print(f"`define N_PINS_WIDTH {math.ceil( math.log2(len(pinname2order)))}",file=f)
    if CLK_PIN_N!="":
        print(f"`define CLK_IS_DIFF ",file=f)
        